Source : https://codeforces.com/gym/103861/problem/A

# A. DFS Order
> 각 테스트 별 제한 시간 : 3초.
>
> 각 테스트 별 제한 메모리 크기 : 1024mb
>
> 입출력 : standard I/O

# 1. 문제 요약 정리

</br>

 $n$개의 node가 존재하는 rooted tree에서 아래의 pseudo-code에 의한 DFS order로 각 노드를 순회할 때, 특정 노드에 방문하는 순서의 최솟값과 최대값을 구하여라.

---

```
let dfs_order be an empty list

def dfs(vertex x):
    append x to the end of dfs_order.
    for (each son y of x): // sons can be iterated in arbitrary order.
        dfs(y)

dfs(root)
```
---


## 입력

첫 줄에는 테스트 케이스의 수를 지정하는 하나의 정수 $T \; (1 \leq T \leq 10^6)$.

각 테스트 케이스마다 (두 번째 줄 이후부터) 첫 줄에는 정수 $n \; (1 \leq n \leq 10^5)$. 그리고 이후 다음 각 $n-1$의 줄에서는 두 정수 $x, y$가 표기되며 이 때 $x$는 $y$의 parent node $( 1\leq x,y \leq n).$ 이런 두 노드들로 형성되는 간선들은 1 노드를 root로 하는 tree를 형성한다.

각 테스트 케이스에 대응하는 $n$의 총합은 $10^6$이 넘지 않는다.

## 출력

각 테스트 케이스마다, $n$개의 결과물을 각 줄마다 출력하며, 이렇게 출력하는 $i$번째 줄에는 $i$번째 노드의 방문 순서의 최솟값과 최대값을 출력한다.

## Example
---
### **input**
```
2
4
1 2
2 3
3 4
5
1 2
2 3
2 4
1 5
```
---
### **output**
```
1 1
2 2
3 3
4 4
1 1
2 3
3 5
3 5
2 5
```

# 2. 문제 해결

### 해결 사고 과정

## (1) 최소값
 이는 간단하다. 의사 코드에서 child node의 순회를 임의로 진행하기 때문에 각 노드의 최솟값은 해당 노드의 Level 값일 것이다.


## (2) 최댓값을 구하기 위해 생각해본 방법
 ###     1) BF 에 가까운 방법
 
 각 노드의 최대값에 대해 Dynamic programming을 활용. </br>
 
 BFS 방식으로 단계별로 최대값을 증가시키면서, 노드를 방문할 때 현재 순서 값이 노드에 저장된 최대값 이하일 때 컷하면 되지 않을까? </br>
 구상 단계에서는 뭔가 그럴듯 한거 같았는 데 실제로 그것이 정확히 어떻게 동작하는 것이냐.를 정리하지 못해서 다른 방법을 찾아봄

 ### 2) 점화식 구하기
 특정 노드의 Max 값은 자신을 포함한 ancestor 들의 모든 silbing들을 다 탐색한 다음 순서값이라는 사실이었다. 그것을 수식화하면 아래와 같아진다.
> $i$ 번째 node의 순서의 최댓값을 $M_i$,
>
> $i$의 parant를 $i^p$,
>
> $i$의 degree를 $d_i$,
>
> $i$의 $j$ 번째 sibling을 $i^s_j$ $(1\leq j \leq d_{i^p})$,
>
> $i$를 root로 하는 sub-tree 내부의 순서의 최대값을 $M^T_i$이라 하자
>
> 그러면 $M_i$ = $M_{i^p}$ + $\sum_{k=0}^{d_{i^p}} M^T_{i^s_k}$ + 1 이 된다.

정리해놓고 나니 무언가 재귀적으로 해결될 것 같아서 좀 들여다봤다. 그런데 계속보다보니 실행 시간에 문제가 있을 것 같다는 생각이 들었다. 우항의 두번째 term은 dfs로도 해결할 수 있다. 

```

def dfs_2(vertex x):

    current_p = length of dfs_order                  # check current position

    append x to the end of dfs_order.  
    for (each son y of x): // sons can be iterated in arbitrary order.
        dfs(y)

    max_sub_tree = length of dfs_order - current_p   # calculate how many nodes is added 
    x.max_sub_tree = max_sub_tree

```

이후 구한 각각의 sub_tree의 max 값을 각 노드에 저장하면 우항의 두번째 term은 구할 수 있으나 문제는 dfs의 한 순회에서 parent 노드의 순서의 최대값을 안다는 것이 보장되지 않는다는 점이었다. 그 parent의 최댓값은 또한 그 parent의 sibling과 parent에 의존한다. 즉 재귀적으로 특정 노드의 첫 번째 term을 구하기 위해서는, ancestor들의 max 값을 모두 알아야한다. 따라서 1회의 dfs를 모두 끝내어 각 노드의 max_sub_tree 값을 구한 뒤에 다시 답을 구하기 위해서 순회를 한 번 진행해야한다. (sibling 노드들을 참조해야하기 때문에 bfs 방식이 좋지 않을까 싶다)

 ### 3) 최종 해결
 
 두 번 순회하면 time out이 날 것이 예상이 되기 때문에 더 좋은 방법이 있나 생각했다. 그러다가 아주 당연한 것을 눈치챘다. 그것은 즉 2번 방법에서 구한 **$M^T_i$의 값이 해당 서브 트리에 존재하는 노드의 갯수**라는 사실이다.
 
 > $M^T_i$ = $|T_i|$
 
 이 때 $|T_i|$는 $i$를 root로 하는 sub-tree의 cardinality이다. 그리고 항상 leaf node의 최댓값은 tree가 가지는 순서 값의 최댓값인 점을 통해 $M_i$를 다음과 같이 구할 수 있다.
 
 > $M_i = n - |T_i| + 1$ 
 
 즉, **구하고자하는 최대값은 노드의 수에서 해당 노드의 descendant의 수를 뺀 값**이다.
 
 그런데 앞서서 descendant를 구하는 내용을 (2)의 pseudo code에 작성해 두었다. 이를 반영하여 최솟값과 최대값을 구하는 pseudo code를 다음과 같이 작성할 수 있다.
 
 ```

def dfs_3(vertex x, depth d):

    x.min_value = d
    
    append x to the end of dfs_order.  
    current_p = length of dfs_order                  # check current position

    for (each son y of x): // sons can be iterated in arbitrary order.
        dfs(y, d+1)

    noDescendant = length of dfs_order - current_p   # calculate how many nodes is added
    
    x.max_value = n - noDescendant

```

## 코드 변천사 [Python & C++]

 기본적인 원리는 실제로 제출했던 python script과 c++ cpp file 모두 유사하다.

## [Version 1.0]
### 이하의 코드는 notebook에서 구동을 위한 코드.

In [2]:
x = [2,
[4,
[1, 2],
[2, 3],
[3, 4]],
[5,
[1, 2],
[2, 3],
[2, 4],
[1, 5]]]

In [3]:
class Tree:
    def __init__(self, data):
        self.children = []
        self.min_value = float("nan")
        self.max_value = -1
def make_graph(x, n):
    vertex = [Tree(i) for i in range(n+1)]
    for parent, child in x:
        vertex[parent].children.append(vertex[child])
    return vertex
        
def dfs(v, d, dfs_list):
    
    v.min_value = d
    dfs_list.append(v)
    
    current_p = len(dfs_list)
    
    for child in v.children:
        dfs(child, d+1, dfs_list)
        
    noDescendant = len(dfs_list) - current_p
    v.max_value = n - noDescendant
    

for i in range(x[0]):
    n = x[1+i][0]
    data = x[1+i][1:]
    dfs_list = []
    vlist = []
    
    vertex = make_graph(data, n)
        
    dfs(vertex[1], 1, dfs_list)
    
    for i in range(1, n+1):
        print(vertex[i].min_value, vertex[i].max_value)
    print("\n")

1 1
2 2
3 3
4 4


1 1
2 3
3 5
3 5
2 5




## [Version 1.1] Python script. A.py -> 실패

Time Out이 발생하여 최대한 시간 소모를 줄일 수 있는 구간을 줄였다.

길이를 통해 하위 노드의 갯수를 구하는 것을 리턴 값을 통해 구하게 하였다. 하지만 그래도 Test #2에서 TimeOut이 발생했다.

In [6]:
class Tree:
    def __init__(self, data):
        self.children = []
        self.min_value =0
        self.max_value =0
         
def dfs(v, d):
    
    v.min_value = d
    
    noDescendant = 0
    
    for child in v.children:
        noDescendant += dfs(child, d+1)
        
    v.max_value = n - noDescendant
    return noDescendant + 1
    

T = int(input()) # The number of Test case


for i in range(T):
    n = int(input())
 
    #make_graph
    vertex = [Tree(i) for i in range(n+1)]
    for j in range(n-1):
        parent, child = list(map(int, input().split()))
        vertex[parent].children.append(vertex[child])

    dfs(vertex[1], 1)
    print("Answer") 
    for i in range(1, n+1):
        print(vertex[i].min_value, vertex[i].max_value)

2
4
1 2
2 3
3 4
Answer
1 1
2 2
3 3
4 4
5
1 2
2 3
2 4
1 5
Answer
1 1
2 3
3 5
3 5
2 5


## [Version 1.2] C++  -> 실패

문제를 해결하지 못한 것이 알고리즘 문제인지 언어의 태생적인 문제인지 알아야했다.

그래서 python code를 C++로 바꿔서 진행했다.

```c++

#include <iostream>
#include <vector>
#include <cstdio>

using namespace std;

struct Node{
    /* data */
    int min_value;
    int max_value;
    vector<int> child;
};

class Tree{
    
    public:
    Node * nodes;
   
    Tree(int n){
        nodes = new Node[n + 1];
    }
};


int dfs(Tree & tree, int i, int depth, int n){

    tree.nodes[i].min_value = depth;
    int noDesc = 0;
    for (auto x : tree.nodes[i].child){
        noDesc += dfs(tree, x, depth+1, n);
    }

    tree.nodes[i].max_value = n - noDesc;
    return noDesc + 1 ; 
}

int main(){

    int t; // Test case
    int n; // n
    int parent, child;
    vector<pair<int,int>> edge;
    cin >> t;
    
    while ( t > 0 ){
        cin >> n;
        Tree tree(n);

        for (int i=0; i<n-1; i++){
            cin >> parent >> child;
            edge.push_back(make_pair(parent, child));
            }
        for (auto x : edge){
                tree.nodes[x.first].child.push_back(x.second);
        }
        dfs(tree, 1, 1, n);

        for (int i=1; i< n+1; i++){
            cout<<tree.nodes[i].min_value << " " <<tree.nodes[i].max_value<<endl;
        }
        edge.clear();
        t--;
    }

    return 0;
}

```

코드의 전반적인 부분은 python과 거의 동일하다. 하지만 그간 몰랐던 사실이 있었는데

>std::vector 함수는 값을 입력할 때 값을 copy해 넣는 사실을 몰라서 헤맸다.

node 구조체의 child가 이전에는 node 구조체 벡터였는 데, Tree에 할당된 node를 구조체 내부의 벡터 구조체에 넣으니 DFS에서 Max 값과 Min값이  child에는 반영이 되지만 Tree의 node 행렬에는 반영되지 않았다. 이 것 때문에 너무 오랜 시간을 잡아먹었다.

하지만 이래도 TimeOut의 고비는 벗어나질 못했다.

## [Version 1.3] C++ -> 성공 (+7)

time loss의 주요 원인인 입출력 문제를 해결하여 성공했다. 

```c++
#include <vector>
#include <cstdio>

using namespace std;

struct Node
{
    /* data */
    int min_value;
    int max_value;
    vector<int> child;
};


class Tree{
    
    public:
    Node * nodes;
   
    Tree(int n){
        nodes = new Node[n + 1];
    }
};

int dfs(Tree & tree, int i, int depth, int n){

    tree.nodes[i].min_value = depth;
    int noDesc = 0;
    for (auto x : tree.nodes[i].child){
        noDesc += dfs(tree, x, depth+1, n);
    }

    tree.nodes[i].max_value = n - noDesc;
    return noDesc + 1 ; 
}

int main(){

    int t; // Test case
    int n; // n
    int parent, child;
    vector<pair<int,int>> edge;
    scanf("%d", &t);

    while ( t > 0 ){
        // cin >> n;
        scanf("%d", &n);
        Tree tree(n);

        for (int i=0; i<n-1; i++){
            scanf("%d %d", &parent, &child);
            edge.push_back(make_pair(parent, child));
            }
        for (auto x : edge){
                tree.nodes[x.first].child.push_back(x.second);
        }
        dfs(tree, 1, 1, n);

        for (int i=1; i< n+1; i++){
            printf("%d %d\n", tree.nodes[i].min_value, tree.nodes[i].max_value);
        }
        edge.clear();
        t--;
    }

    return 0;
}
```

기존의 Standard I/O로 사용되는 cin과 cout 대신에 c style의 scanf와 printf를 사용하니 바로 성공했다.